In [1]:
import random, gc, os, pickle, csv, time

import datasets.utils
import models.utils
from models.cls_oml_adapter import OML
from models.base_models_ori import LabelAwareReplayMemory

import numpy as np

import higher
import torch
import torch.nn.functional as F
from torch.utils import data

/root/MetaLifelongLanguage/env-adapter/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Constants

In [2]:
dataset_order_mapping = {
    1: [2, 0, 3, 1, 4],
    2: [3, 4, 0, 1, 2],
    3: [2, 4, 1, 3, 0],
    4: [0, 2, 1, 4, 3]
}
n_classes = 33
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
# model_path = "/data/model_runs/original_oml/aOML-order1-2022-07-18/OML-order1-id4-2022-07-18_17-53-13.518612.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr002-2022-07-31/OML-order1-id4-2022-07-31_14-53-46.456804.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr005-2022-07-31/OML-order1-id4-2022-07-31_18-47-41.477968.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr005-up20-2022-08-01/OML-order1-id4-2022-08-01_14-45-55.869765.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-07-31/OML-order1-id4-2022-07-31_21-18-36.241546.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr020-2022-08-16/OML-order1-id4-2022-08-16_11-37-19.424113.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr050-2022-08-16/OML-order1-id4-2022-08-16_14-16-12.167637.pt"

# v. SR 
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-08-29-sr/OML-order1-id4-2022-08-29_18-10-31.695669.pt"
# v. SR Query
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-08-30-sr-query/OML-order1-id4-2022-08-30_05-21-18.854228.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-14-21-sreversed/OML-order1-id4-2022-12-15_03-55-41.428520.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2023-01-03-srev-replay/OML-order1-id4-2023-01-03_02-59-30.904898.pt"
# model_path = "/data/model_runs/original_oml/aOML-order2-2023-01-07-srev-replay/OML-order2-id4-2023-01-07_19-01-27.774382.pt"
# model_path = "/data/model_runs/original_oml/aOML-order3-2023-01-08-srev-replay/OML-order3-id4-2023-01-08_12-03-38.782801.pt"
# model_path = "/data/model_runs/original_oml/aOML-order4-2023-01-08-srev-replay/OML-order4-id4-2023-01-08_21-01-31.303061.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2023-01-20-srev-replay2/OML-order1-id4-2023-01-21_19-49-04.497477.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2023-02-13-srev-curri/OML-order1-id4-2023-02-13_05-46-13.147873.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr005-2023-02-23-lorav2/OML-order1-id4-2023-02-23_23-07-12.342518.pt"
model_path = "/data/model_runs/original_oml/aOML-order1-inlr005-2023-02-26-lorav5/OML-order1-id4-2023-02-26_05-50-49.468357.pt"


# memory_path = "/data/model_runs/original_oml/aOML-order1-2022-07-18/OML-order1-id4-2022-07-18_17-53-13.518639_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr002-2022-07-31/OML-order1-id4-2022-07-31_14-53-46.456828_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr005-2022-07-31/OML-order1-id4-2022-07-31_18-47-41.477992_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr005-up20-2022-08-01/OML-order1-id4-2022-08-01_14-45-55.869797_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-07-31/OML-order1-id4-2022-07-31_21-18-36.241572_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr020-2022-08-16/OML-order1-id4-2022-08-16_11-37-19.424139_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr050-2022-08-16/OML-order1-id4-2022-08-16_14-16-12.167666_memory.pickle"
# v. SR 
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-08-29-sr/OML-order1-id4-2022-08-29_18-10-31.695692_memory.pickle"
# v. SR Query
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-08-30-sr-query/OML-order1-id4-2022-08-30_05-21-18.854254_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-14-21-sreversed/OML-order1-id4-2022-12-15_03-55-41.428551_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2023-01-03-srev-replay/OML-order1-id4-2023-01-03_02-59-30.904925_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order2-2023-01-07-srev-replay/OML-order2-id4-2023-01-07_19-01-27.774413_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order3-2023-01-08-srev-replay/OML-order3-id4-2023-01-08_12-03-38.782827_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order4-2023-01-08-srev-replay/OML-order4-id4-2023-01-08_21-01-31.303088_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2023-01-20-srev-replay2/OML-order1-id4-2023-01-21_19-49-04.497510_memory.pickle"
memory_path = "/data/model_runs/original_oml/aOML-order1-inlr005-2023-02-26-lorav5/OML-order1-id4-2023-02-26_05-50-49.468380_memory.pickle"


# new_memory_path, ext = os.path.splitext(memory_path)
# new_memory_path = new_memory_path + "_label" + ext

use_db_cache = True
cache_dir = 'tmp'

In [3]:
args = {
    "order": 1,
    "n_epochs": 1,
    "lr": 3e-5,
    "inner_lr": 0.001*2,
    "inner_adapter_lr": 1e-4*20,
    "meta_lr": 3e-5,
    "model": "roberta",
    "learner": "oml",
    "mini_batch_size": 16,
    "updates": 5*1,
    "write_prob": 0.1,
    "max_length": 448,
    "seed": 42,
    "replay_rate": 0.01,
    "replay_every": 8000,
    "task_aware": True,
    "reverse_support": True,
    "curriculum_replay": True,
    "pln": "1fc",
    "adapter": True
}
sort_score = True
updates = args["updates"]
mini_batch_size = args["mini_batch_size"]
order = args["order"]

In [4]:
torch.manual_seed(args["seed"])
random.seed(args["seed"])
np.random.seed(args["seed"])

# Load Dataset

In [5]:
print('Loading the datasets')
test_datasets = []
for dataset_id in dataset_order_mapping[order]:
    test_dataset_file = os.path.join(cache_dir, f"{dataset_id}.cache")
    if os.path.exists(test_dataset_file):
        with open(test_dataset_file, 'rb') as f:
            test_dataset = pickle.load(f)
    else:
        test_dataset = datasets.utils.get_dataset_test("", dataset_id)
        print('Loaded {}'.format(test_dataset.__class__.__name__))
        test_dataset = datasets.utils.offset_labels(test_dataset)
        pickle.dump(test_dataset, open( test_dataset_file, "wb" ), protocol=pickle.HIGHEST_PROTOCOL)
        print(f"Pickle saved at {test_dataset_file}")
    test_datasets.append(test_dataset)
print('Finished loading all the datasets')

Loading the datasets
Finished loading all the datasets


# Load Model

In [6]:
learner = OML(device=device, n_classes=n_classes, **args)
print('Using {} as learner'.format(learner.__class__.__name__))
learner.load_model(model_path)
with open(memory_path, 'rb') as f:
#     learner.memory = pickle.load(f)
    memory_buffer = pickle.load(f)


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Name                     Architecture         #Param      %Param  Active   Train
--------------------------------------------------------------------------------
lora                     bottleneck          894,528       0.718       1       1
--------------------------------------------------------------------------------
Full model                               124,645,632     100.000               1


2023-02-26 08:06:34,668 - OML-Log - INFO - Loaded TransformerRLN as RLN
2023-02-26 08:06:34,673 - OML-Log - INFO - Loaded LinearPLN as PLN
/root/MetaLifelongLanguage/env-adapter/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Using OML as learner


In [7]:
# Setting up task dict for task-aware
memory_buffer.task_dict = {
    0: list(range(5, 9)), # AG
    1: list(range(0, 5)), # Amazon
    2: list(range(0, 5)), # Yelp
    3: list(range(9, 23)), # DBPedia
    4: list(range(23, 33)), # Yahoo
}

In [8]:
dataclass_mapper = {
    "AGNewsDataset": 0,
    "AmazonDataset": 1,
    "YelpDataset": 2,
    "DBPediaDataset": 3,
    "YahooAnswersDataset": 4
}
dataclass_mapper["AGNewsDataset"]

0

# Testing

Select specific column index per row
https://stackoverflow.com/questions/23435782/numpy-selecting-specific-column-index-per-row-by-using-a-list-of-indexes

In [9]:
def evaluate(dataloader, updates, mini_batch_size, dataname=""):
    learner.rln.train()
    learner.pln.train()
    # Enable adapter training (Freeze all Others that isn't LoRA)
    learner.rln.encoder.train_adapter("lora")

    all_losses, all_predictions, all_labels, all_label_conf = [], [], [], []
    all_adaptation_time = []
    # Get Query set first. and then find supporting support set
    for query_idx, (query_text, query_labels) in enumerate(dataloader):
        print(f"Query ID {query_idx}/{len(dataloader)}")
        # The task id to optimize to for support set
        # task_idx = get_task_from_label_list(query_labels, memory_buffer.task_dict)
        task_idx = dataclass_mapper[dataname]
        
    
        support_set = []
        for _ in range(updates):
            text, labels = memory_buffer.read_batch_task(batch_size=mini_batch_size, task_idx=task_idx, sort_score=sort_score)
            support_set.append((text, labels))

        with higher.innerloop_ctx(learner.combinedRLNPLN, learner.inner_optimizer,
                                  copy_initial_weights=False, track_higher_grads=False) as (frlnpln, diffopt):
            
            INNER_tic = time.time()
            # Inner loop
            task_predictions, task_labels = [], []
            support_loss = []
            for text, labels in support_set:
                labels = torch.tensor(labels).to(device)
                input_dict = learner.rln.encode_text(text)
                output = frlnpln(input_dict)
                loss = learner.loss_fn(output, labels)
                diffopt.step(loss)
                pred = models.utils.make_prediction(output.detach())
                support_loss.append(loss.item())
                task_predictions.extend(pred.tolist())
                task_labels.extend(labels.tolist())
            INNER_toc = time.time() - INNER_tic
            all_adaptation_time.append(INNER_toc)

            acc, prec, rec, f1 = models.utils.calculate_metrics(task_predictions, task_labels)

            print('Support set metrics: Loss = {:.4f}, accuracy = {:.4f}, precision = {:.4f}, '
                        'recall = {:.4f}, F1 score = {:.4f}'.format(np.mean(support_loss), acc, prec, rec, f1))

            # Query set is now here!
            query_labels = torch.tensor(query_labels).to(device)
            query_input_dict = learner.rln.encode_text(query_text)
            with torch.no_grad():
                query_output = frlnpln(query_input_dict) # Output has size of torch.Size([16, 33]) [BATCH, CLASSES]
                query_loss = learner.loss_fn(query_output, query_labels)
            query_loss = query_loss.item()
            # print(output.detach().size())
            # output.detach().max(-1) max on each Batch, which will return [0] max, [1] indices
            query_output_softmax = F.softmax(query_output, -1)
            query_label_conf = query_output_softmax[np.arange(len(query_output_softmax)), query_labels] # Select labels in the softmax of 33 classes

            query_pred = models.utils.make_prediction(query_output.detach())
            query_acc, query_prec, query_rec, query_f1 = models.utils.calculate_metrics(query_pred.tolist(), query_labels.tolist())
            
            print('Query set metrics: Loss = {:.4f}, accuracy = {:.4f}, precision = {:.4f}, '
                'recall = {:.4f}, F1 score = {:.4f}'.format(np.mean(query_loss), query_acc, query_prec, query_rec, query_f1))

            all_losses.append(query_loss)
            all_predictions.extend(query_pred.tolist())
            all_labels.extend(query_labels.tolist())
            all_label_conf.extend(query_label_conf.tolist())

    acc, prec, rec, f1 = models.utils.calculate_metrics(all_predictions, all_labels)
    print('Test metrics: Loss = {:.4f}, accuracy = {:.4f}, precision = {:.4f}, recall = {:.4f}, '
                'F1 score = {:.4f}'.format(np.mean(all_losses), acc, prec, rec, f1))
    return acc, prec, rec, f1, all_predictions, all_labels, all_label_conf, all_adaptation_time

In [10]:
tic = time.time()
print('----------Testing on test set starts here----------')

accuracies, precisions, recalls, f1s = [], [], [], []
all_adapt_time = []
# Data for Visualization: [data_idx, label, label_conf, pred]
data_for_visual = []

for test_dataset in test_datasets:
    print('Testing on {}'.format(test_dataset.__class__.__name__))
    test_dataloader = data.DataLoader(test_dataset, batch_size=mini_batch_size, shuffle=False,
                                      collate_fn=datasets.utils.batch_encode)
    acc, prec, rec, f1, all_pred, all_label, all_label_conf, all_adaptation_time = evaluate(dataloader=test_dataloader, updates=updates, 
                                                mini_batch_size=mini_batch_size, dataname=test_dataset.__class__.__name__)
    
    data_ids = [test_dataset.__class__.__name__ + str(i) for i in range(len(all_label))]
    data_for_visual.extend(list(zip(data_ids, all_label, all_label_conf, all_pred)))
    all_adapt_time.extend(all_adaptation_time)
    
    accuracies.append(acc)
    precisions.append(prec)
    recalls.append(rec)
    f1s.append(f1)


print()
print("COPY PASTA - not really but ok")
for row in accuracies:
    print(row)
print()
print('Overall test metrics: Accuracy = {:.4f}, precision = {:.4f}, recall = {:.4f}, '
            'F1 score = {:.4f}'.format(np.mean(accuracies), np.mean(precisions), np.mean(recalls), np.mean(f1s)))

toc = time.time() - tic
print(f"Total Time used: {toc//60} minutes")

----------Testing on test set starts here----------
Testing on YelpDataset
Query ID 0/475
Support set metrics: Loss = 0.3192, accuracy = 0.9125, precision = 0.9267, recall = 0.9100, F1 score = 0.9145
Query set metrics: Loss = 0.7459, accuracy = 0.7500, precision = 0.7933, recall = 0.7600, F1 score = 0.7467
Query ID 1/475
Support set metrics: Loss = 0.4907, accuracy = 0.8250, precision = 0.8304, recall = 0.8167, F1 score = 0.8197
Query set metrics: Loss = 0.8936, accuracy = 0.7500, precision = 0.6933, recall = 0.7933, F1 score = 0.7133
Query ID 2/475
Support set metrics: Loss = 0.4787, accuracy = 0.9000, precision = 0.9454, recall = 0.9033, F1 score = 0.9161
Query set metrics: Loss = 0.8937, accuracy = 0.6875, precision = 0.5433, recall = 0.7000, F1 score = 0.6092
Query ID 3/475
Support set metrics: Loss = 0.3947, accuracy = 0.8875, precision = 0.9046, recall = 0.8833, F1 score = 0.8872
Query set metrics: Loss = 1.3506, accuracy = 0.6250, precision = 0.6733, recall = 0.6000, F1 score = 

Support set metrics: Loss = 0.3978, accuracy = 0.8500, precision = 0.8710, recall = 0.8433, F1 score = 0.8477
Query set metrics: Loss = 0.9267, accuracy = 0.6250, precision = 0.7000, recall = 0.6375, F1 score = 0.6643
Query ID 36/475
Support set metrics: Loss = 0.3467, accuracy = 0.8875, precision = 0.8885, recall = 0.8800, F1 score = 0.8810
Query set metrics: Loss = 0.7817, accuracy = 0.6250, precision = 0.6167, recall = 0.6333, F1 score = 0.5552
Query ID 37/475
Support set metrics: Loss = 0.3616, accuracy = 0.8375, precision = 0.8424, recall = 0.8267, F1 score = 0.8230
Query set metrics: Loss = 0.9415, accuracy = 0.5625, precision = 0.4167, recall = 0.4433, F1 score = 0.4167
Query ID 38/475
Support set metrics: Loss = 0.3668, accuracy = 0.9125, precision = 0.9236, recall = 0.9100, F1 score = 0.9146
Query set metrics: Loss = 1.0375, accuracy = 0.6250, precision = 0.5100, recall = 0.6100, F1 score = 0.5500
Query ID 39/475
Support set metrics: Loss = 0.3343, accuracy = 0.9000, precision

Query set metrics: Loss = 1.3923, accuracy = 0.5000, precision = 0.5800, recall = 0.6267, F1 score = 0.5822
Query ID 71/475
Support set metrics: Loss = 0.3929, accuracy = 0.9000, precision = 0.9129, recall = 0.8967, F1 score = 0.9034
Query set metrics: Loss = 0.8172, accuracy = 0.7500, precision = 0.6600, recall = 0.6500, F1 score = 0.6325
Query ID 72/475
Support set metrics: Loss = 0.4192, accuracy = 0.8500, precision = 0.8675, recall = 0.8433, F1 score = 0.8471
Query set metrics: Loss = 0.9415, accuracy = 0.5625, precision = 0.5762, recall = 0.6900, F1 score = 0.5810
Query ID 73/475
Support set metrics: Loss = 0.4634, accuracy = 0.8500, precision = 0.8795, recall = 0.8467, F1 score = 0.8557
Query set metrics: Loss = 0.9774, accuracy = 0.6875, precision = 0.6033, recall = 0.6200, F1 score = 0.5976
Query ID 74/475
Support set metrics: Loss = 0.4859, accuracy = 0.8625, precision = 0.8897, recall = 0.8600, F1 score = 0.8688
Query set metrics: Loss = 1.1957, accuracy = 0.6250, precision =

Support set metrics: Loss = 0.3990, accuracy = 0.8750, precision = 0.8858, recall = 0.8700, F1 score = 0.8697
Query set metrics: Loss = 1.0462, accuracy = 0.3750, precision = 0.3800, recall = 0.4667, F1 score = 0.4003
Query ID 107/475
Support set metrics: Loss = 0.5028, accuracy = 0.8500, precision = 0.8913, recall = 0.8500, F1 score = 0.8608
Query set metrics: Loss = 0.6525, accuracy = 0.8125, precision = 0.8667, recall = 0.8167, F1 score = 0.7914
Query ID 108/475
Support set metrics: Loss = 0.5607, accuracy = 0.8375, precision = 0.8903, recall = 0.8400, F1 score = 0.8602
Query set metrics: Loss = 1.0125, accuracy = 0.4375, precision = 0.3800, recall = 0.3967, F1 score = 0.3714
Query ID 109/475
Support set metrics: Loss = 0.4937, accuracy = 0.8125, precision = 0.8374, recall = 0.8033, F1 score = 0.7971
Query set metrics: Loss = 1.2129, accuracy = 0.6250, precision = 0.6733, recall = 0.6367, F1 score = 0.6267
Query ID 110/475
Support set metrics: Loss = 0.3668, accuracy = 0.8875, preci

Support set metrics: Loss = 0.3372, accuracy = 0.8625, precision = 0.8686, recall = 0.8533, F1 score = 0.8554
Query set metrics: Loss = 0.7348, accuracy = 0.6875, precision = 0.5600, recall = 0.6000, F1 score = 0.5778
Query ID 142/475
Support set metrics: Loss = 0.3227, accuracy = 0.9125, precision = 0.9208, recall = 0.9067, F1 score = 0.9057
Query set metrics: Loss = 0.5468, accuracy = 0.8125, precision = 0.6800, recall = 0.7500, F1 score = 0.7111
Query ID 143/475
Support set metrics: Loss = 0.4085, accuracy = 0.8500, precision = 0.8571, recall = 0.8400, F1 score = 0.8384
Query set metrics: Loss = 0.6235, accuracy = 0.6250, precision = 0.7033, recall = 0.6700, F1 score = 0.6267
Query ID 144/475
Support set metrics: Loss = 0.3079, accuracy = 0.8875, precision = 0.8897, recall = 0.8833, F1 score = 0.8822
Query set metrics: Loss = 1.4785, accuracy = 0.5625, precision = 0.4500, recall = 0.6500, F1 score = 0.5010
Query ID 145/475
Support set metrics: Loss = 0.4488, accuracy = 0.8500, preci

...

Support set metrics: Loss = 0.5099, accuracy = 0.8625, precision = 0.8493, recall = 0.8600, F1 score = 0.8479
Query set metrics: Loss = 0.5552, accuracy = 0.8750, precision = 0.9062, recall = 0.9062, F1 score = 0.8810
Query ID 313/475
Support set metrics: Loss = 0.4328, accuracy = 0.9250, precision = 0.9026, recall = 0.9100, F1 score = 0.9032
Query set metrics: Loss = 0.5388, accuracy = 0.8125, precision = 0.8542, recall = 0.9083, F1 score = 0.8604
Query ID 314/475
Support set metrics: Loss = 0.4630, accuracy = 0.9000, precision = 0.9328, recall = 0.9000, F1 score = 0.9089
Query set metrics: Loss = 0.3970, accuracy = 0.9375, precision = 0.8214, recall = 0.8571, F1 score = 0.8367
Query ID 315/475
Support set metrics: Loss = 0.4167, accuracy = 0.9375, precision = 0.9476, recall = 0.9300, F1 score = 0.9315
Query set metrics: Loss = 0.7513, accuracy = 0.7500, precision = 0.9286, recall = 0.7619, F1 score = 0.7965
Query ID 316/475
Support set metrics: Loss = 0.4862, accuracy = 0.9125, preci

Support set metrics: Loss = 0.4468, accuracy = 0.9000, precision = 0.9065, recall = 0.9200, F1 score = 0.9061
Query set metrics: Loss = 0.5899, accuracy = 0.8125, precision = 0.8125, recall = 0.7500, F1 score = 0.7708
Query ID 348/475
Support set metrics: Loss = 0.5234, accuracy = 0.8375, precision = 0.8571, recall = 0.8300, F1 score = 0.8182
Query set metrics: Loss = 0.8929, accuracy = 0.6875, precision = 0.5833, recall = 0.6111, F1 score = 0.5630
Query ID 349/475
Support set metrics: Loss = 0.4583, accuracy = 0.9000, precision = 0.8990, recall = 0.9000, F1 score = 0.8940
Query set metrics: Loss = 0.7283, accuracy = 0.8125, precision = 0.9286, recall = 0.8810, F1 score = 0.8762
Query ID 350/475
Support set metrics: Loss = 0.5023, accuracy = 0.8500, precision = 0.8454, recall = 0.8300, F1 score = 0.8276
Query set metrics: Loss = 0.3943, accuracy = 0.8750, precision = 1.0000, recall = 0.8958, F1 score = 0.9333
Query ID 351/475
Support set metrics: Loss = 0.4849, accuracy = 0.9000, preci

Support set metrics: Loss = 0.4938, accuracy = 0.8750, precision = 0.8813, recall = 0.8600, F1 score = 0.8645
Query set metrics: Loss = 0.8177, accuracy = 0.8125, precision = 0.7619, recall = 0.7143, F1 score = 0.6905
Query ID 383/475
Support set metrics: Loss = 0.3706, accuracy = 0.9625, precision = 0.9718, recall = 0.9600, F1 score = 0.9641
Query set metrics: Loss = 0.5353, accuracy = 0.8750, precision = 0.9286, recall = 0.9238, F1 score = 0.9079
Query ID 384/475
Support set metrics: Loss = 0.4208, accuracy = 0.9250, precision = 0.9292, recall = 0.9100, F1 score = 0.9164
Query set metrics: Loss = 1.2817, accuracy = 0.6250, precision = 0.5833, recall = 0.6500, F1 score = 0.5633
Query ID 385/475
Support set metrics: Loss = 0.4816, accuracy = 0.9000, precision = 0.9130, recall = 0.9000, F1 score = 0.9010
Query set metrics: Loss = 0.8886, accuracy = 0.7500, precision = 0.8125, recall = 0.7396, F1 score = 0.7488
Query ID 386/475
Support set metrics: Loss = 0.4523, accuracy = 0.8875, preci

Support set metrics: Loss = 0.4590, accuracy = 0.9500, precision = 0.9376, recall = 0.9500, F1 score = 0.9407
Query set metrics: Loss = 0.6124, accuracy = 0.8125, precision = 0.8000, recall = 0.8250, F1 score = 0.7857
Query ID 418/475
Support set metrics: Loss = 0.4454, accuracy = 0.9000, precision = 0.9065, recall = 0.9000, F1 score = 0.8954
Query set metrics: Loss = 0.7799, accuracy = 0.8125, precision = 0.9167, recall = 0.8646, F1 score = 0.8530
Query ID 419/475
Support set metrics: Loss = 0.4237, accuracy = 0.9125, precision = 0.9365, recall = 0.8900, F1 score = 0.9040
Query set metrics: Loss = 1.1150, accuracy = 0.5000, precision = 0.6250, recall = 0.4583, F1 score = 0.5000
Query ID 420/475
Support set metrics: Loss = 0.4131, accuracy = 0.9000, precision = 0.9042, recall = 0.8900, F1 score = 0.8927
Query set metrics: Loss = 1.0177, accuracy = 0.6875, precision = 0.5000, recall = 0.6111, F1 score = 0.5185
Query ID 421/475
Support set metrics: Loss = 0.4294, accuracy = 0.9250, preci

Support set metrics: Loss = 0.5996, accuracy = 0.8375, precision = 0.8414, recall = 0.8200, F1 score = 0.8211
Query set metrics: Loss = 0.2018, accuracy = 1.0000, precision = 1.0000, recall = 1.0000, F1 score = 1.0000
Query ID 453/475
Support set metrics: Loss = 0.5010, accuracy = 0.9125, precision = 0.9303, recall = 0.9100, F1 score = 0.9145
Query set metrics: Loss = 0.5860, accuracy = 0.7500, precision = 0.8958, recall = 0.8021, F1 score = 0.8155
Query ID 454/475
Support set metrics: Loss = 0.5168, accuracy = 0.8750, precision = 0.8856, recall = 0.8800, F1 score = 0.8775
Query set metrics: Loss = 0.4262, accuracy = 0.8750, precision = 0.8000, recall = 0.8500, F1 score = 0.8167
Query ID 455/475
Support set metrics: Loss = 0.4750, accuracy = 0.8750, precision = 0.8999, recall = 0.8800, F1 score = 0.8843
Query set metrics: Loss = 0.6061, accuracy = 0.7500, precision = 0.5481, recall = 0.6296, F1 score = 0.5796
Query ID 456/475
Support set metrics: Loss = 0.4903, accuracy = 0.9125, preci

In [11]:
_model_path0 = os.path.splitext(model_path)[0]
csv_filename = _model_path0 + "_update"+ str(updates) +"_results.csv" # for selective replay
with open(csv_filename, 'w') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(["data_idx", "label", "label_conf", "pred"])
    csv_writer.writerows(data_for_visual)
print(f"Done writing CSV File at {csv_filename}")

Done writing CSV File at /data/model_runs/original_oml/aOML-order1-inlr005-2023-02-26-lorav5/OML-order1-id4-2023-02-26_05-50-49.468357_update5_results.csv


In [12]:
# Log Time for Inference
_model_path0 = os.path.splitext(model_path)[0]
time_txt_filename = _model_path0 + "_update"+ str(updates) +"_time_inference.csv" 
with open(time_txt_filename, 'w') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(["time_id", "time"])
    csv_writer.writerow(["Total Time", f"{toc//60} minutes"])
    csv_writer.writerow(["mean Adapt Time", f"{np.mean(all_adapt_time)} s"])
print(f"Done writing Time CSV File at {time_txt_filename}")

Done writing Time CSV File at /data/model_runs/original_oml/aOML-order1-inlr005-2023-02-26-lorav5/OML-order1-id4-2023-02-26_05-50-49.468357_update5_time_inference.csv
